# **Exercise Sheet 3: Causality**

In [1]:
from scipy import stats
import numpy as np

## **Exercise 3-1** *Granger causal test*
Consider the following dataset from the R package “lmtest” to answer the age old question of what came first,
“the chicken or the egg”. The data was presented by Walter Thurman and Mark Fisher in the American Journal
of Agricultural Economics, May 1988, titled “Chickens, Eggs, and Causality, or Which Came First?”

year | chicken (Y) | egg (X)
---- | ----------- | -------
1930 | 468491      | 3581
1931 | 449743      | 3532
1932 | 436815      | 3327
1933 | 444523      | 3255
1934 | 433937      | 3156
1935 | 389958      | 3081
<p style="text-align:center;">Tabelle 1: Population of chickens and eggs in U.S. egg production.</p>
In this example, we will use only 6 entries of our data (1930 - 1935) and a single lag of 1.

### **a)** *Auto-regression for X and Y*
The auto regression problem can be written in the matrix format as:
$$\begin{pmatrix} 
    Y (2) \\
    . \\
    . \\
    . \\
    Y (n)
\end{pmatrix} = \beta_0 \begin{pmatrix} 
    1 \\
    . \\
    . \\
    . \\
    1
\end{pmatrix} + \beta_1 \begin{pmatrix} 
    Y (1) \\
    . \\
    . \\
    . \\
    Y (n-1)
\end{pmatrix} + \beta_2 \begin{pmatrix} 
    X (1) \\
    . \\
    . \\
    . \\
    X (n-1)
\end{pmatrix}$$

The design matrix X for this case is:

$$\begin{pmatrix} 
    1 & Y (1) & X (1)\\
    . & . & . \\
    . & . & . \\
    . & . & . \\
    1 & Y (n-1) & X (n-1)
\end{pmatrix}$$

Calculate the unknown coefficients $\beta = (\beta_0, \beta_1, \beta_2)$, by using the derived formula, discussed in our
lecture:

$$\hat{\beta} =(X^{'}X)^+X^{'}Y$$

where $X^{'}$
is the transpose of the matrix $X$ and $X^+$ is the generalised inverse. Calculate $\lVert e \rVert ^2_2$
, the sum of
squared residuals for the first regression which is our $RSS_1$.

In [2]:
def calc_rss(Y,Y_pred):
    return np.sum(np.square(Y[1:] - Y_pred))
def calc_beta_hat(X,Y):
    return np.linalg.pinv(X[:-1,:].T @ X[:-1]) @ X[:-1,:].T @ Y[1:]
def pred_y(X,beta_hat):
    return X[:-1,:] @ beta_hat

In [3]:
X = np.array([
    [1,468491,3581],
    [1,449743,3532],
    [1,436815,3327],
    [1,444523,3255],
    [1,433937,3156],
    [1,389958,3081]
    ])
X_t = np.copy(X.T)
X_plus = np.linalg.pinv(X)
Y = np.copy(X[:,1])

In [4]:
X

array([[     1, 468491,   3581],
       [     1, 449743,   3532],
       [     1, 436815,   3327],
       [     1, 444523,   3255],
       [     1, 433937,   3156],
       [     1, 389958,   3081]])

In [5]:
beta_hat = calc_beta_hat(X,Y)

In [6]:
Y_pred_1 = pred_y(X,beta_hat)

In [7]:
RSS_1 = calc_rss(Y,Y_pred_1)

In [8]:
beta_hat

array([ 1.20111918e+05, -6.90431823e-02,  1.01396058e+02])

In [9]:
RSS_1

np.float64(1023104023.6108718)

### **b)** *Auto-regression without Y* 
Repeat the steps from the previous task to calculate $RSS_2$ (i. e. the sum of squared residuals for the
second regression), with the 2nd design matrix, which does not contain the egg feature, corresponding to
the 2nd Granger equation.

In [10]:
X2 = np.delete(X,2,axis=1)  
X2_t = np.copy(X2.T)
X2_plus = np.linalg.pinv(X2)

In [11]:
X2

array([[     1, 468491],
       [     1, 449743],
       [     1, 436815],
       [     1, 444523],
       [     1, 433937],
       [     1, 389958]])

In [12]:
beta_hat_2 = calc_beta_hat(X=X2,Y=Y)

In [13]:
Y_pred_2 = pred_y(X=X2,beta_hat=beta_hat_2)

In [14]:
RSS_2 = calc_rss(Y=Y,Y_pred=Y_pred_2)

In [15]:
beta_hat_2

array([-5.17190454e+04,  1.08061854e+00])

In [16]:
RSS_2

np.float64(1385892406.5564852)

### **c)** *Apply statistical test* 
Apply the Granger Sargent test with $\alpha = 0.05$, to the computed $RSS_1$ and $RSS_2$ values, to test the
Granger causal direction from eggs to chicken. We use a single lag of 1, i.e. $d = 1$. Interpret the result.

In [17]:
GS = ((RSS_2 - RSS_1)/1)/(RSS_2/(6 - 2*1))

In [36]:
GS

np.float64(1.0470896044434808)

In [18]:
alpha = 0.05

In [19]:
p_value = stats.f.sf(GS,dfn=1,dfd=(6 - 2*1))

In [37]:
p_value

np.float64(0.36402477901824704)

In [20]:
if p_value <= alpha:
    print("Null hypothesis rejected: The egg caused chickens to be born")
else:
    print("Null hypothesis not rejected: The egg does not have to cause chickens to be born")

Null hypothesis not rejected: The egg does not have to cause chickens to be born


### **d)** *Causal test for direction from chicken to eggs*
Do exercises a - c analogously for the equations:
$$\begin{pmatrix} 
    X (2) \\
    . \\
    . \\
    . \\
    X (n)
\end{pmatrix} = \beta_0 \begin{pmatrix} 
    1 \\
    . \\
    . \\
    . \\
    1
\end{pmatrix} + \beta_1 \begin{pmatrix} 
    X (1) \\
    . \\
    . \\
    . \\
    X (n-1)
\end{pmatrix} + \beta_2 \begin{pmatrix} 
    Y (1) \\
    . \\
    . \\
    . \\
    Y (n-1)
\end{pmatrix}$$
And interpret the results, what they mean for the answer ”what was first, eggs or chicken?”.

In [21]:
X_hat = np.array([
    [1,3581,468491],
    [1,3532,449743],
    [1,3327,436815],
    [1,3255,444523],
    [1,3156,433937],
    [1,3081,389958]
    ])

In [22]:
beta_hat_3 = calc_beta_hat(X=X_hat,Y=X_hat[:,1])

In [23]:
X_pred_1 = pred_y(X=X_hat,beta_hat=beta_hat_3)

In [24]:
RSS_1_hat= calc_rss(Y=X_hat[:,1],Y_pred=X_pred_1)

In [25]:
beta_hat_3

array([-9.41423511e+02,  5.71643330e-01,  5.11542859e-03])

In [26]:
RSS_1_hat

np.float64(8511.149774128166)

In [27]:
X2_hat = np.delete(X_hat,2,axis=1)  

In [28]:
beta_hat_4 = calc_beta_hat(X=X2_hat,Y=X2_hat[:,1])

In [29]:
X_pred_2 = pred_y(X=X2_hat,beta_hat=beta_hat_4)

In [30]:
RSS_2_hat= calc_rss(Y=X_hat[:,1], Y_pred=X_pred_2)

In [31]:
beta_hat_4

array([230.73506161,   0.90186486])

In [32]:
RSS_2_hat

np.float64(13768.29021504951)

In [33]:
GS_hat = ((RSS_2_hat - RSS_1_hat)/1)/(RSS_2_hat/(6 - 2*1))

In [38]:
GS_hat

np.float64(1.5273183115140896)

In [34]:
p_value_hat = stats.f.sf(GS_hat,dfn=1,dfd=(6 - 2*1))

In [39]:
p_value_hat

np.float64(0.2841313442120876)

In [35]:
if p_value_hat <= alpha:
    print("Null hypothesis rejected: The chicken caused eggs to be born")
else:
    print("Null hypothesis not rejected: The chicken does not have to cause eggs to be born")

Null hypothesis not rejected: The chicken does not have to cause eggs to be born


## **Exercise 3-2** *Multivariate Granger model*

### **a)** *Consider graphical Granger model and causal inference by ordinary least squares with adaptive lasso penalty and regularization parameter $\lambda_n = n^{\frac{3}{2}}$. Is the problem solved by this model consistent? Explain your answer in detail.*

No, the problem solved is not consistent, since one of the oracle properties for that to be the case has to be: $\frac{\lambda_n}{\sqrt{n}}\rightarrow 0$

But: $\frac{n^{\frac{3}{2}}}{\sqrt{n}} = \frac{\sqrt{n^3}}{\sqrt{n}} = \sqrt{\frac{n^3}{n}} = \sqrt{n^2} \neq 0$

Therefore the problem solved is not consistent.

### **b)** *What is the algorithm HMMLGA for and what are its hyperparameters?*

The algorithm HMMLGA is for calculating a causal adjacency matrix for a given GGM.

The hyperparameters are: 
$$n_g\dots Maximum\ iterations$$
$$m\dots Population\ size\ (even)$$

## **Exercise 3-3** *Bivariate causal models on non-temporal data*

### **a)** *Why is the causal relationship between X and Y by bivariate additive noise models for the linear-Gaussian case non-identifiable? Explain.*

Because the model induces the same joint distribution on X and Y, making it impossible to calculate $E_Y$ and $E_X$, because they have cyclic dependencies for $\mu$ and $\sigma^2$.
$$\mu_{E_X} = (1-\alpha\beta)\mu_X - \beta\mu_{E_Y}$$
$$\sigma^2_{E_X} = (1-\alpha\beta)^2\sigma^1_X+\beta^2\sigma^2_{E_Y}$$

### **b)** *Recall the example 1 from lecture 4:*
$$solar\ cell\ (cause) \rightarrow generation\ of\ electricity\ (effect)$$

One can change $P (cause)$ without affecting $P (effect|cause)$: A change of intensity/angle of sun changes
the power output of the cell, but not the conditional distribution of the effect given the cause. One can
change $P (effect|cause)$ without affecting $P (cause)$: 

E.g. by using more efficient cells - while this changes
the power output of cells, it does not affect the distribution of the incoming radiation.
To do the same in the anti-causal direction is hard: i.e. to find actions changing only $P(effect)$ and not
$P (cause|effect)$ or vice versa, as due to their causal connection they are intrinsically (more) dependent
on each other.

Now let us have a pair of variable $X= age$ of a person and $Y = diastolic\ blood\ pressure$, see e.g. pair 0040
in Mooij, J.M., Peters, J., Janzing, D., Zscheischler, J., Schölkopf, B. Distinguishing Cause from Effect
Using Observational Data: Methods and Benchmarks. JMLR 17(32):1-102, 2016.

What is the causal relation between $X$ and $Y$? How one could change $P (effect|cause)$ without affecting
$P (cause)$ in the correct causal direction in this concrete case? Describe, how it would be in the anti-causal
direction.

$X$ causes $Y$. 

In this concrete case where $X= age$ and $Y = diastolic\ blood\ pressure$, one could change $P (Y|X)$ without affecting $P (X)$ by having the participants take blood pressure reducing medication.

In the anti-causal direction, one would have to change the age, based on the blood pressure, which is actually impossible, because blood pressure cannot alter time.